[Liger Kenrel](https://github.com/linkedin/Liger-Kernel) &mdash; это библиотека с оптимизированными кернелами для торча, написанными на triton, которые часто применяются в большх современных моделях. Засчёт того, что мы спускаемся на уровень абстракций ниже, где можно более деталеьно усправлять памятью и вычислениями, получается сэкономить до 60% памяти и ускорить модели на примерно 20% в некоторых случаях.

На данный момент оптимизированные кернелы есть для следующих слоёв: RoPE, RMSNorm, SwiGLU, CrossEntropyLoss и других.

In [1]:
import gc
import random
from os import environ
environ["CUDA_VISIBLE_DEVICES"] = "GPU-fe2d8dfd-06f2-a5c4-a7fd-4a5f23947005"
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from liger_kernel.transformers import AutoLigerKernelForCausalLM, LigerCrossEntropyLoss, LigerFusedLinearCrossEntropyLoss

In [2]:
# Seeding everything for deterministic results

seed = 138

random.seed(seed)
environ["PYTHONHASHSEED"] = str(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

Создадим даталоадер для маленького датасета со школьными задачами. Токенизатор используем стандартный для модели.

In [3]:
class ListSentenceDataset(Dataset):
    def __init__(self, sentence_list):
        self.data = sentence_list

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx: int):
        return self.data[idx]
    
    def __getitems(self, idx: slice):
        return self.data[idx]

ds = ListSentenceDataset(load_dataset("madrylab/gsm8k-platinum")["test"]["question"])

tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B", padding_side="left", truncation_side="left", return_tensors="pt")
def encoding_collator(batch):
    global tokenizer
    return tokenizer(batch, padding=True, return_tensors="pt")

BS = 12
train_dl = DataLoader(ds, batch_size=BS, shuffle=False, collate_fn=encoding_collator)
eval_dl = DataLoader(ds, batch_size=BS * 3, shuffle=False, collate_fn=encoding_collator)

Функции для тестирования обучения модели.

In [4]:
DEVICE = torch.device("cuda:0")

def train_loop(model, train_loader, criterion, optimizer, num_epochs: int = 2, device = DEVICE, use_fused_kernel: bool = False):
    model.train()
    for _ in tqdm(range(num_epochs)):
        iter_pbar = tqdm(train_loader, leave=False)
        for batch in iter_pbar:
            input_ids, attn_mask = batch["input_ids"].to(device), batch["attention_mask"].to(device)

            if use_fused_kernel:
                outputs = model(input_ids = input_ids[:, :-1], attention_mask = attn_mask[:, :-1], output_hidden_states=True).hidden_states[-1].flatten(0, -2)
                target = input_ids[:, 1:].reshape(-1)
                loss = criterion(model.lm_head.weight, outputs, target)
            else:
                outputs = model(input_ids = input_ids[:, :-1], attention_mask = attn_mask[:, :-1]).logits.flatten(0, -2)
                target = input_ids[:, 1:].reshape(-1)
                loss = criterion(outputs, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def test_training(model, criterion, device=DEVICE, use_fused_kernel: bool = False):
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=6e-4)

    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.reset_peak_memory_stats()

    train_loop(model=model, train_loader=train_dl, criterion=criterion, optimizer=optimizer, device=device, use_fused_kernel=use_fused_kernel)
    del model, criterion, optimizer
    print(f"Max memory allocated: {torch.cuda.max_memory_allocated() // (2 ** 20)} MB")

Прогон без Liger Kernel.

In [5]:
model = AutoModelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B", torch_dtype="bfloat16")
criterion = nn.CrossEntropyLoss()
test_training(model, criterion)

100%|██████████| 2/2 [01:21<00:00, 40.57s/it]

Max memory allocated: 14502 MB


Включим Liger Kernel и прогоним эпоху ещё раз.

In [6]:
model = AutoLigerKernelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B", torch_dtype="bfloat16")
criterion = LigerCrossEntropyLoss()
test_training(model, criterion)

100%|██████████| 2/2 [01:11<00:00, 35.68s/it]

Max memory allocated: 13174 MB


Как мы видим, получилось неольшое ускорение и небольшое сокращение памяти. На больших батччах и моделях этот эффект будет ещё более заметен.

Ещё хочу скзаать, что если использовать лосс, посчитанный напрямую в модели, можно выиграть ещё больше, так как в Liger Kernels существуют лоссы, совмещающие в себе реальный лосс и линейный слой перед ним, что позволяет экономить ещё больше памяти. Однако с таким интерфейсом вычисления происходят дольше и в целях демонстрации я выбрал тот подход, который написан выше. Ячейку ниже можно запустить, елси хочется перепроверить получаемый максимальный объём памяти.

In [7]:
model = AutoLigerKernelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B", torch_dtype="bfloat16")
criterion = LigerFusedLinearCrossEntropyLoss()
test_training(model, criterion, use_fused_kernel=True)

100%|██████████| 2/2 [01:53<00:00, 56.63s/it]

Max memory allocated: 12911 MB


Теперь протестируем поведение моделей на инференсе.

In [8]:
DEVICE = torch.device("cuda:0")

def eval_loop(model, eval_loader, num_epochs: int = 4, device = DEVICE):
    model.eval()
    with torch.no_grad():
        for _ in tqdm(range(num_epochs)):
            iter_pbar = tqdm(eval_loader, leave=False)
            for batch in iter_pbar:
                input_ids, attn_mask = batch["input_ids"].to(device), batch["attention_mask"].to(device)
                outputs = model(input_ids = input_ids, attention_mask = attn_mask)

def test_eval(model, device=DEVICE):
    model.to(device)

    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.reset_peak_memory_stats()

    eval_loop(model=model, eval_loader=eval_dl, device=device)
    del model
    print(f"Max memory allocated: {torch.cuda.max_memory_allocated() // (2 ** 20)} MB")

Прогоним сначала обычную версию, а потом с Liger Kernel.

In [9]:
model = AutoModelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B", torch_dtype="bfloat16")
test_eval(model)

100%|██████████| 4/4 [00:30<00:00,  7.51s/it]

Max memory allocated: 5455 MB


In [10]:
model = AutoLigerKernelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B", torch_dtype="bfloat16")
test_eval(model)

100%|██████████| 4/4 [00:30<00:00,  7.53s/it]

Max memory allocated: 5455 MB


На инференсе эффект от использования Liger Kernel менее заметен, потому что задача более простая и не требует бэквардов.